## ________________________________________________________________________
## Quantum Problem Mapping: Transverse Field Ising Model (TFIM)
## _Updated for Qiskit 2.0 - Using New Reference Primitives_

In [14]:
#Import essential libraries for Qiskit
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit.primitives import StatevectorSampler, StatevectorEstimator # New primitives.
from qiskit.visualization import plot_histogram, plot_state_city
import numpy as np

# ________________________________________________________________________
# Step 1: Map Problem to Circuits and Observables

In [15]:
# Define TFIM parameters
num_qubits =4
J = 1.0 #Interaction strength
h = 0.5 #Transverse field strength
trotter_steps = 3 # Time evolution steps
shots = 5000

# build time evolution circuit
def build_tfim_circuit(num_qubits, J, h, trotter_steps):
    """
    Constructs quantum circuit for TFIM time evolution using Trotterization

    Physics Insight:
    TFIM Hamiltonian: H = -J Sum Z_i Z_{i+1} -h Sum X_i
    Trotter decomposition: e^{-iHt} ≈ [e^{-iH_zz δt} e^{-iH_x δt}]^N
    """
    qc = QuantumCircuit(num_qubits, name ="TFIM Evolution")

    # Time evolution for each Trotter step
    for step in range(trotter_steps):
        #ZZ interaction terms
        for i in range(num_qubits -1):
            qc.cx(i,i+1)
            qc.rz(2*J/trotter_steps, i+1) # Angle adjusted for Trotter steps
            qc.cx(i,i+1)

        #Transverse field terms
        #Physics: e^{-iH_x δt}]^N
        for i in range(num_qubits):
            qc.rx(2*h/trotter_steps, i) #Rx rotation implements X term

    return qc
#Create the evolution circuit ( no measurements for Estimator)
tfim_circuit = build_tfim_circuit(num_qubits, J,h,trotter_steps)

#Create measurement circuit for Sampler
tfim_circuit_with_measurement = tfim_circuit.copy()
tfim_circuit_with_measurement.measure_all()

    
    

# ________________________________________________________________________
# Step 2: Define Observables for Qiskit 2.0

In [19]:
"""
Observable Definition in Qiskit 2.0:
- All observables must be SparsePauliOp objects
- String notation supported: "ZZII", "XIXI", etc.
- Coefficients can be complex numbers
"""

#Define magnetization observable (single-site)
magnetization = SparsePauliOp("IIIZ")/num_qubits

#Define full TFIM Hamiltonian for energy calculation
hamiltonian_terms = []
coefficients = []

#ZZ interaction terms
for i in range(num_qubits -1):
    pauli_string = "I"*i + "ZZ" + "I"*(num_qubits -i -2)
    hamiltonian_terms.append(pauli_string)
    coefficients.append(-J)


# X field terms
for i in range(num_qubits):
    pauli_string = "I"*i + "X" +"I"*(num_qubits - i -1)
    hamiltonian_terms.append(pauli_string)
    coefficients.append(-h)
#Construct full hamiltonian
full_hamiltonian = SparsePauliOp(hamiltonian_terms,coefficients)

# ________________________________________________________________________
# Step 3: Execute with Qiskit 2.0 primitives

In [26]:
"""
Execution Model in Qiskit 2.0:
1. StatevectorSampler: For measurement sampling
2. StatevectorEstimator: For expectation value computation
3. PUB format: (circuit, [observables], [parameters])
"""
#Initialize
sampler = StatevectorSampler(default_shots=shots, seed=42)
estimator = StatevectorEstimator(default_precision=0.01, seed=42)

#Execute with StatevectorSampler (PUB format)
print("Executing with StatevectorSampler...")
sampler_pub = (tfim_circuit_with_measurement,) # PUB tuple
sampler_job = sampler.run([sampler_pub])
sampler_result = sampler_job.result()

#Access measurement results from the new data structure
counts = sampler_result[0].data.meas.get_counts() # meas is default register name

#Execute with StatevectorEstimator (PUB format)
print("Executing with StatevectorEstimator...")
estimator_pub = (tfim_circuit, [magnetization]) # PUB with observables
magnetization_job = estimator.run([estimator_pub])
magnetization_result = magnetization_job.result()
mag_expectation = magnetization_result[0].data.evs[0]

#Calculate full energy
energy_pub = (tfim_circuit, [full_hamiltonian]) 
energy_job = estimator.run([energy_pub])
energy_result = energy_job.result()
energy_expectation = energy_result[0].data.evs[0]








Executing with StatevectorSampler...
Executing with StatevectorEstimator...


# ________________________________________________________________________
# Step 4: Advanced Analysis with Statevector

In [27]:
#Get exact statevector for comparison
statevector = Statevector(tfim_circuit)

#Compute exact expectation values
exact_magnetization = statevector.expectation_value(magnetization).real
exact_energy = statevector.expectation_value(full_hamiltonian).real

# ________________________________________________________________________
# Step 5: Error Mitigation and Post-Processing

In [28]:
def apply_readout_corrections(counts,error_rate=0.02):

    corrected_counts ={}
    for bitstring, count in counts.items():
        #Apply correction based on error model
        corrected_count = count*(1-error_rate)**len(bitstring)
        corrected_counts[bitstring] = int(corrected_count)

#Apply corrections
corrected_counts = apply_readout_corrections(counts)

        

# ________________________________________________________________________
# Visualization and Results

In [29]:

# Plot measurement distribution
plot_histogram(counts, title="TFIM Measurement Results (Qiskit 2.0)")

# Plot statevector
plot_state_city(statevector, title="TFIM Quantum State")

# Print comprehensive results
print("\n" + "="*60)
print("TFIM SIMULATION RESULTS (Qiskit 2.0)")
print("="*60)
print(f"System parameters:")
print(f"  - Qubits: {num_qubits}")
print(f"  - Interaction strength J: {J}")
print(f"  - Transverse field h: {h}")
print(f"  - h/J ratio: {h/J:.2f}")
print(f"  - Trotter steps: {trotter_steps}")
print()
print(f"Magnetization results:")
print(f"  - Exact (statevector): {exact_magnetization:.6f}")
print(f"  - Estimated (primitive): {mag_expectation:.6f}")
print(f"  - Theoretical (h/J={h/J:.1f}): {0.82 if h/J < 1 else 0.18:.6f}")
print()
print(f"Energy results:")
print(f"  - Exact energy: {exact_energy:.6f}")
print(f"  - Estimated energy: {energy_expectation:.6f}")
print()
print(f"Quantum phase analysis:")
if h/J < 1:
    print(f"  - System in ORDERED phase (ferromagnetic)")
    print(f"  - Expected high magnetization")
else:
    print(f"  - System in DISORDERED phase (paramagnetic)")
    print(f"  - Expected low magnetization")



TFIM SIMULATION RESULTS (Qiskit 2.0)
System parameters:
  - Qubits: 4
  - Interaction strength J: 1.0
  - Transverse field h: 0.5
  - h/J ratio: 0.50
  - Trotter steps: 3

Magnetization results:
  - Exact (statevector): 0.165247
  - Estimated (primitive): 0.168295
  - Theoretical (h/J=0.5): 0.820000

Energy results:
  - Exact energy: -2.876543
  - Estimated energy: -2.873496

Quantum phase analysis:
  - System in ORDERED phase (ferromagnetic)
  - Expected high magnetization


# ________________________________________________________________________
# Advanced: Parameter Sweep Example